# Exploring spatial and biodiversity data

```{admonition} Summary
:class: hint
This walkthrough provides a complete data processing example from start to finish. We demonstrate how to combine spatial and biodiversity data in a practical example.

The example is based on two data sources:
- **Biodiversity data** from [Lebendiger Atlas der Natur Deutschlands (LAND)](https://land.gbif.de/): Species observations as coordinate-based vector data.
- **Spatial data** from [IOER Monitor](https://www.ioer-monitor.de/): Aggregated and processed land use data that is provided in raster format (WCS).

```

## Lebendiger Atlas der Natur Deutschlands (LAND)

[LAND - Lebendiger Atlas der Natur Deutschlands](https://land.gbif.de/) compiles species observation data in Germany. It displays selected occurrence datasets that are publicly available through the [Global Biodiversity Information Facility (GBIF)](Gbif.org). GBIF aims to provide open access to biodiversity data worldwide. This connection enables users to explore Germany’s biodiversity data via LAND while also accessing and utilizing selected datasets from GBIF’s global database.

There are two ways to find and download data:
- **Via web browser**: Search, explore and download individual data files directly from [LAND](https://land.gbif.de/) using its occurrence search, geo viewer, and download options.
- **Via code**: Use the [GBIF Application Programming Interface (API) Reference](https://techdocs.gbif.org/en/openapi/) to access data programmatically.

For both methods, and especially for larger downloads, registration is required.

```{figure} https://images.naturalis.nl/original/csr%2FD80_8153.jpg
:name: sparrow-graphic

English Sparrow, *Passer domesticus* (GBIF.org).
```

```{admonition} Using GBIF API
:class: warning
 

```

## IOER Monitor of Settlement and Open Space Development (IOER Monitor)
The IOER Monitor of Settlement and Open Space Development (short [IOER Monitor](https://monitor.ioer.de/)) is a research data infrastructure provided by the Leibniz Institute of Ecological Urban and Regional Development (IOER). It offers insights into land use structure, development, and landscape quality in Germany. Indicators and data can be explored and visualized in an [interactive geo viewer](https://monitor.ioer.de).
All IOER Monitor data is available through **Web Feature Service (WFS)** and **Web Coverage Service (WCS)**, allowing users to retrieve spatial data in standardized formats.

There are two ways to access and download data:

- **Via web browser**: Login, search and download individual data files directly from the IOER Monitor's download services.
- **Via code**: Use the Monitor API to access data programmatically.

In both cases, [registration](https://monitor.ioer.de/monitor_api/) is required.

```{admonition} Using the Monitor API
:class: warning

If you want to use the Monitor API, you need to register:

1. Register at [monitor.ioer.de/monitor_api/signup](https://monitor.ioer.de/monitor_api/signup).
2. Generate a personal API key in your account settings.
3. Store your personal API key in a file called `.env` as `API_KEY=xyz`
```

 ```{figure} https://www.ioer-monitor.de/fileadmin/user_upload/monitor/img/Ergebnisse/siedlungsdichte.png
:name: monitor-graphic

Wie dicht leben wir? [IOER Monitor data](https://www.ioer-monitor.de/ergebnisse/analyseergebnisse/wie-dicht-leben-wir/).
```

## Example research question

In the next section, we demonstrate how to combine spatial and biodiversity data in a practical example.

Specifically, we will investigate whether there is a positive correlation between *Passer domesticus* (common name: `House Sparrow`, `English Sparrow`) density and settlement areas. Since house sparrows are known to prefer urban environments, this pattern should be visible using data from the IOER Monitor and LAND data.

Additionally, we will examine population trends for house sparrows in Saxony. According to {cite:ts}`salek_house_2015`, an overall decline in house sparrow populations has been observed:

> _Populations of House and Tree Sparrows have rapidly declined in various breeding habitats throughout their European distribution range; however, the strongest decline was recorded within urban environments._
{cite:p}`salek_house_2015`

But is this also true for Saxony? [Let’s explore whether the data supports this trend...](202_data_retrieval)

```{admonition} Summary
:class: hint
While Python offers multiple ways to process data, we will focus on a single approach without exploring alternative routes. Where further details are useful, cross-references to the reference documentation (Part III) or external sources are provided.
```

## References

```{bibliography}
:style: unsrt
:filter: docname in docnames
```